<a href="https://colab.research.google.com/github/ryakkalauncc/3162project4/blob/main/ITCS3162Project4Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Instructions: For Project 4, the goal is to get some experience working with clustering.

You have the freedom to define your own problem and choose a dataset. However, your problem definition needs to relate to clustering.

The writing will be critical here, as the main goal will be to discuss your process, why you take certain steps (e.g., what preprocessing steps and why), and tell a story around your data and insights gained through both modeling (working with the clustering algorithms we learned about) and visualizations. Thus, your writing should portray your critical thinking about the data, the process, and what knowledge you find.

See the rubric for more information on the parts required.
The project should be completed as a part of your portfolio and have all the required parts.


**Introduce the problem**
Introduce the project. What is the problem you are trying to solve? What questions are you trying to find answers to?

**For this project, it is required that you use clustering in some way.**

**What is clustering and how does it work?**
Explain what clustering is and how it works (e.g., k-means and/or agglomerative that we have gone over in class).

**Introduce the data**
Where did you find it? What is the data about (include links)? What are the features (with additional explanations if not already self-explanatory from the name itself)?

**Data Understanding/Visualization**
Use methods to try to further understand and visualize the data. Make sure to remember your initial problems/questions when completing this step.
While exploring, does anything else stand out to you (perhaps any surprising insights?)

**How does this step relate to your modeling?**

**Pre-processing the data**
What pre-processing steps do you follow? Explain why you do each pre-processing step.


**Modeling (Clustering)**
What model(s) do you use to try to solve your problem? Why do you choose those model(s)?

For example, why choose k-means over agglomerative, or vice versa? Or perhaps experiment with both and discuss the pros/cons of each? You may also try experimenting with other methods of clustering not discussed in class.

**Storytelling (Clustering Analysis)**
Use this section to further analyze your clusters.

What information or insights does it tell you? What have you learned? Were you able to answer your initial problems/questions (if so, discuss that)?

**Impact Section**
Discuss the possible impact of your project. This can be socially, ethically, etc. It cannot be something like "our project has no impact" or "our project has no negative impact." Even the most well-intentioned projects *could* have a negative impact. We will not be checking for "right" or "wrong" answers, but showing your critical thinking.

**References**
Include any references.